In [9]:
spark = SparkSession.builder \
    .appName("records") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()


ConnectionRefusedError: [Errno 111] Connection refused

In [2]:
!pip install pandas xlsxwriter



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
spark.sql("DROP TABLE IF EXISTS demo.nyc.taxis_10M_50COLUMNS_batch2")


DataFrame[]

In [4]:
import os
iceberg_table_dir = "../warehouse/nyc/taxis_10M_50COLUMNS_batch2"
metadata_dir = f"{iceberg_table_dir}/metadata"
data_dir = f"{iceberg_table_dir}/data"
input_data_dir = f"../input_data"
analysis_info = []
records_before_op = 0

def append_to_file(file_path, msg):
    open_mode = "a"
    if not os.path.exists(file_path):
        open_mode = "w"

    # Open the CSV file in write mode
    with open(file_path, open_mode) as file:
        writer = csv.writer(file)
        
        if open_mode=="w":
            #writing header of the columns
            writer.writerows([list(msg.keys())])    

        row_values = [list(msg.values())]
        # Write the data to the CSV file
        writer.writerows(row_values)

def get_size():
    # List the metadata files
    manifest_pattern = re.compile(r".*-m\d+\.avro$")
    metadata_files = os.listdir(metadata_dir)
    
    # Initialize variables to store the sizes of different types of metadata files
    snap_avro_size = 0
    metadata_json_size = 0
    m_avro_size = 0

    data_dir_size = 0
    # get data dir size
    data_dir_files = os.listdir(data_dir)
    # print(data_dir_files)
    for filename in data_dir_files:
        file_path = os.path.join(data_dir, filename)
        data_dir_size += os.path.getsize(file_path) / 1024  # Convert size to KB
    
    # Iterate through the metadata files and calculate their sizes
    for file in metadata_files:
        file_path = os.path.join(metadata_dir, file)
        file_size_kb = os.path.getsize(file_path) / 1024  # Convert size to KB
        
        if file.startswith("snap-") and file.endswith(".avro"):
            snap_avro_size += file_size_kb
        elif file.endswith(".metadata.json"):
            metadata_json_size += file_size_kb
        elif manifest_pattern.match(file):
            m_avro_size += file_size_kb
    
    # Print the time taken and the sizes of the metadata files
    # print(f"Time taken to read Parquet files: {time_taken:.2f} seconds")
    # print(f"Size of snap-*.avro files: {snap_avro_size:.2f} KB")
    # print(f"Size of *.metadata.json files: {metadata_json_size:.2f} KB")
    # print(f"Size of *m{0-9}{1,}.avro files: {m_avro_size:.2f} KB")

    return {"data_dir_size": data_dir_size,"metadata_size": metadata_json_size,"snapshot_size": snap_avro_size,"manifest_size": m_avro_size}


In [5]:
from pyspark.sql.types import (
    DoubleType, FloatType, LongType, StructType, StructField, 
    StringType, IntegerType, DateType
)

# Define the schema with 50 columns based on the required data types
schema = StructType([
    # StructField("vendor_id", LongType(), True),  # INT
    # StructField("trip_id", LongType(), True),  # INT
    # StructField("trip_distance", FloatType(), True),  # FLOAT
    # StructField("fare_amount", DoubleType(), True),  # DOUBLE
    # StructField("store_and_fwd_flag", StringType(), True)  # STRING
# ] + [
    # Assigning VARCHAR, INT, STRING, and DATE data types in a cyclic pattern
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 0 else  # VARCHAR
    StructField(f"extra_col_{i}", IntegerType(), True) if i % 4 == 1 else  # INT
    StructField(f"extra_col_{i}", StringType(), True) if i % 4 == 2 else  # STRING
    StructField(f"extra_col_{i}", DateType(), True)  # DATE
    for i in range(50)
])

# Create an empty DataFrame with the schema
df = spark.createDataFrame([], schema)

# Create the Iceberg table
df.writeTo("demo.nyc.taxis_10M_50COLUMNS_batch2").create()


25/03/03 07:20:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
df = spark.table("demo.nyc.taxis_10M_50COLUMNS_batch2")
df.show()

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|extra_col_0|extra_col_1|extra_col_2|extra_col_3|extra_col_4|extra_col_5|extra_col_6|extra_col_7|extra_col_8|extra_col_9|extra_col_10|extra_col_11|extra_col_12|extra_col_13|extra_col_14|extra_col_15|extra_col_16|extra_col_17|extra_col_18|extra_col_19|extra_col_20|extra_col_21|extra_col_22|extra_col_23|extra_col_24|extra_col_25|extra_col_26|extra_col_27|ext

In [8]:
import time
import csv
from pyspark.sql import functions as F
import os
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("YourApp") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

input_data_dir = "../input_data"
output_dir = "../output"
analysis_info = []
records_before_op = 0
total_insertion_time = 0

file_type = input("Enter input file type csv or parquet? : ").lower().strip()
input_data_dir = os.path.join(input_data_dir, file_type)
input_files = os.listdir(input_data_dir)

analysis_file = os.path.join(output_dir, f"analysis_info_{file_type}.csv")
if os.path.exists(analysis_file):
    os.remove(analysis_file)

# Read the existing Spark table
df = spark.table("demo.nyc.taxis_10M_50COLUMNS_batch2")
records_before_op = df.count()

# Define batch size (e.g., 100,000 records per batch)
batch_size = 100000

for file in input_files:
    print(f"Started with file={file}")
    file_path = os.path.join(input_data_dir, file)

    st = time.time()
    if file_type == "parquet":
        # Read the parquet file in batches
        input_df = spark.read.parquet(file_path)
    else:
        input_df = spark.read.csv(file_path, header=True)
        input_df = input_df.select(
            F.col("extra_col_0").cast("long"),
            F.col("extra_col_1").cast("int"),
            F.col("extra_col_2").cast("string"),
            F.col("extra_col_3").cast("date"),
            *[F.col(f"extra_col_{i}").cast("string" if i % 4 == 0 or i % 4 == 2 else "int" if i % 4 == 1 else "date") for i in range(4, 45)]
        )
    
    rows = input_df.count()
    
    # Processing in batches
    num_batches = (rows // batch_size) + 1
    for batch_num in range(num_batches):
        batch_start = batch_num * batch_size
        batch_end = (batch_num + 1) * batch_size
        
        # Get the batch of records using a range filter
        batch_df = input_df.filter(F.col("extra_col_0").between(batch_start, batch_end))

        # Perform operations on this batch (e.g., append to the table)
        batch_df.writeTo("demo.nyc.taxis_10M_50COLUMNS_batch2").append()

        # Track insertion time for this batch
        end = time.time() - st
        total_insertion_time += end
        records_before_op += batch_df.count()

        print(f"Inserted {batch_df.count()} records from batch {batch_num+1}/{num_batches} in {end:.2f} sec.")
    
# **PRINT INSERTION TIME BEFORE UPDATE**
print(f"\nTotal insertion time: {total_insertion_time:.2f} sec\n")

# === Get Update Percentage from User ===
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100

df = spark.table("demo.nyc.taxis_10M_50COLUMNS_batch2")
total_rows = df.count()

num_rows = int(total_rows * update_percentage)
print(f"Updating {num_rows} rows (~{update_percentage*100}%)...")

# Sample random rows ensuring unique IDs
sampled_df = df.select("extra_col_0").distinct().orderBy(F.rand()).limit(num_rows)
sampled_ids = sampled_df.rdd.map(lambda x: x[0]).collect()

st = time.time()

# Updating `extra_col_1` (integer)
updated_df = df.withColumn(
    "extra_col_1",
    F.when(F.col("extra_col_0").isin(sampled_ids), F.col("extra_col_1") + 10)
    .otherwise(F.col("extra_col_1"))
)

# Overwrite updated data in batches
updated_df.writeTo("demo.nyc.taxis_10M_50COLUMNS_batch2").overwritePartitions()

end = time.time() - st

print(f"Updated {num_rows} rows in {end:.2f} sec")


ConnectionRefusedError: [Errno 111] Connection refused